# generate function yaml

If you have made a local change to a function, it"s local yaml can be updated as follows:

In [ ]:
import mlrun, os, pandas

mlrun.mlconf.dbpath = "http://mlrun-api:8080"
#mlrun.mlconf.hub_url = "https://raw.githubusercontent.com/mlrun/functions/mlrun-v0.4.6/{name}/function.yaml"
mlrun.mlconf.hub_url = "/User/functions/{name}/function.yaml"

funcs_df = pandas.read_csv("functions-table.csv")

for index, row in funcs_df.iterrows():
    # TODO: loop over function subfolders and extract build info instead of reading file
    params={
        "name"     : row["name"],
        "filename" : os.path.join(row["folder"], row["filename"]),
        "image"    : f"{row['image']}:{row['image_tag']}",
        "kind"     : row["kind"],
        "with_doc" : False}
    func=mlrun.code_to_function(**params)
    func.spec.default_handler = row["handler"]
    func.spec.image_pull_policy = "Always"
    func.spec.build.code_origin = f"/User/functions/{row['name']}/function.yaml"
    func.metadata.categories = [row["categoryA"], row["categoryB"]]
    func.metadata.labels = {"author": "yjb", 
                            "stage": "development"}
    func.export(f"{row['folder']}/function.yaml")
    func.save()